# OpenAI API Demo


In [55]:

import json
from openai import OpenAI
import os
from IPython.display import Markdown
import requests
import boto3


Let's first define client:

In [52]:
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

And let's give chat a role first:


In [53]:
chat = [{"role": "system", "content": 'You are software engineer'}]

The way the API works is that we extend the the chat with user messages and assistant replies:

In [54]:
def question(chat_history, some_question, client=client):
        """We take a chat_history, append a question as a user, then get a reply from the assistant, and append that too

        Args:
            chat_history (list): A list of dictionaries, with each dictionary containing a role and content key
            some_question (string): 
        """
        chat_history.append({"role": "user", "content": some_question})
        reply = client.chat.completions.create(
                model="gpt-4",
                messages=chat
                )
        reply_message = reply.choices[0].message
        chat_history.append({'role': reply_message.role, 'content':reply_message.content})
        display(Markdown(reply_message.content))
        

In [59]:
question(chat, "What is most popular programming language today?")

As of 2021, JavaScript is considered the most popular programming language. However, Python and Java are also widely used and popular. Popularity can depend on the specific sector of software engineering, such as data science (Python) or web development (JavaScript).

In [44]:
question(chat, "Can you give me some stats in terms of numbers too?")

According to the Stack Overflow Developer Survey 2021, the most popular programming languages among professional developers are:

- JavaScript - 64.96%
- Python - 44.01%
- Java - 40.2%
- TypeScript - 28.3%
- C# - 27.39%
- PHP - 23.36%
- C++ - 21.57%
- C - 20.59%

Note that these percentages represent the respondents who use these languages, and as developers often use multiple languages, the percentages will add up to more than 100%. The survey had over 80,000 respondents.

## Call AWS Lambda

We will now move the API calls to cloud, in particular to AWS Lambda. We first create a layer with the dependencies:

### Create a Layer

- `virtualenv env_name_of_choice && source env_name_of_choice/bin/activate`
- Lambda has issues with some native libraries per [this](https://docs.aws.amazon.com/lambda/latest/dg/python-package.html#python-package-native-libraries), so make sure to run this when installing packages:
```bash
pip install --platform manylinux2014_x86_64 --target=package --implementation cp --python-version 3.x --only-binary=:all: --upgrade <package_name> -t ./theEnvFolder/python
```
or use old version of `pydantic`:
```bash
pip install openai==1.10.0 pydantic==1.10.12 -t ./theEnvFolder/python   # there is a bug in later versions of pydantic, 
```

now just zip the environment folder and upload it to AWS Lambda:
```bash
cd theEnvFolder
zip -r layer.zip . 
aws lambda publish-layer-version --layer-name openai_layer --zip-file fileb://layer.zip --compatible-runtimes python3.11 --no-cli-pager
```


### Create Lambda

Create IAM Role that will allow to execute AWS Lambda.

Make sure Lambda Python version matches the version in the Layer above.

Let's use the code from f-on `question`. There are two ways we can call Lambda function and pass the payload directly (so no need for AWS API Gateway):
- using URL
- using boto3 

We'll demo both, but URL might be better when using HTML.

```python
import json
from openai import OpenAI
import os

def lambda_handler(event, context):
    """
    event is a same as chat i.e. list of dictionaries with role and content (in a case when called via URL it is slightly different since it is embedded in the extra layer so we extract body first)
    """
    
    api_key = os.getenv("OPENAI_API_KEY")
    client = OpenAI(api_key=api_key)
    
    if 'body' in event:  # this is needed when calling Lambda via URL, since URL call and boto3 have different event structure
        event = json.loads(json.loads(event.get('body')))
        
    chat_history = event
        
    reply = client.chat.completions.create(
        model="gpt-4",
        messages=chat_history
        )
    
    reply_message = reply.choices[0].message
    chat_history.append({'role': reply_message.role, 'content':reply_message.content})

    return {
        'statusCode': 200,
        'body': chat_history
    }
```



#### URL call

Let's say we have a chat, that we want to call Lambda f-on with:

In [282]:
chat = [{'role': 'system', 'content': 'You are software engineer'},
 {'role': 'user',
  'content': 'What is most popular programming language today?'}]

In [290]:
a = list(range(10))
for i in range()
a[1::2]

[1, 3, 5, 7, 9]

In [293]:
list(range(1, 10, 2))

[1, 3, 5, 7, 9]

We want to pass this chat to a Lambda function and then get back the chat that will have appended the reply from the assistant.

In [283]:
# Your Lambda function's URL
lambda_url = "https://5faukxw75uazcs2zdl4zbvyg2e0lebie.lambda-url.us-west-1.on.aws/"
print("Please wait while GPT4 is done ...")
headers = {'Content-Type': 'application/json'}
r = requests.post(headers=headers, url=lambda_url, json=json.dumps(chat))
response_url = json.loads(r.content.decode('utf-8'))

Please wait while GPT4 is done ...


In [284]:
Markdown(response_url[-1]['content'])

As of now, JavaScript is considered one of the most popular programming languages due to its wide use in web development. However, other notable languages in popularity include Python, Java, and C#. It can vary depending on the sector and application.

Second method is using boto3:

In [285]:
lambda_client = boto3.client('lambda')
function_name = 'myGPT'

# Invoke the Lambda function
r = lambda_client.invoke(
    FunctionName=function_name,
    InvocationType='RequestResponse',  # Use 'Event' for asynchronous execution
    Payload=json.dumps(chat))
response_boto3 = json.loads(r['Payload'].read().decode('utf-8'))['body']


In [289]:
response_boto3

[{'role': 'system', 'content': 'You are software engineer'},
 {'role': 'user',
  'content': 'What is most popular programming language today?'},
 {'role': 'assistant',
  'content': 'As of 2022, the most popular programming language is JavaScript, often followed closely by Python and Java. However, the popularity of a language can vary based on different factors such as the industry, geographical location, and specific tasks or workloads involved.'}]

In [287]:
Markdown(response_boto3[-1]['content'])

As of 2022, the most popular programming language is JavaScript, often followed closely by Python and Java. However, the popularity of a language can vary based on different factors such as the industry, geographical location, and specific tasks or workloads involved.

So we have shown two approaches here. Let's now make a webpage out of this so we can repeatedly call the Lambda function and get the response back.
We will use `streamlit` for this.